In [ ]:
import scanpy as sc

# Only needed for processing
import numpy as np
import pandas as pd
import os 
pd.set_option('display.max_rows', 500)

output = '/home/carsten/alvira_bioinformatics/uterus/data/pilot/240510_contractility_specturm_usm'
os.makedirs(output, exist_ok=True)
sc.set_figure_params(dpi=300, format="png")
sc.settings.figdir = output


In [ ]:
adata = sc.read('/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/scanpy_files/uterus_processed_celltyped.gz.h5ad')
adata


In [ ]:
genes = ['ACTA2','DPP6','KCNMA1','OXTR','NOTCH3','ADGRL3','PDE3A',
         ]

In [ ]:
sc.pl.DotPlot(adata, genes, groupby='Cell Subtype').style(cmap='Reds').legend(width=2).savefig(f'{output}/dotplot_all_celltypes.png',dpi=300)
sc.pl.DotPlot(adata[adata.obs['Cell Subtype'].isin(['Uterine smooth muscle','Vascular smooth muscle'])], genes, groupby='Cell Subtype').style(cmap='Reds',largest_dot=175).legend(width=2).savefig(f'{output}/dotplot_smoothmuscle_celltypes.png',dpi=300) 

In [ ]:
adata.obs['GroupContract'].unique()

In [ ]:
rank_dict = {'TL-ND':3,'TNL-GC':2, 'TNL-BC':1,}

In [ ]:
usm_adata = adata[(adata.obs['Cell Subtype']=='Uterine smooth muscle')&
              (adata.obs['GroupContract'].isin(['TL-ND', 'TNL-BC', 'TNL-GC']))
              ]
usm_adata.obs['Contractility_ranking']= [rank_dict[x] for x in usm_adata.obs['GroupContract']]
usm_df = pd.DataFrame(usm_adata.X.todense(), index=usm_adata.obs_names, columns= usm_adata.var_names)
# usm_df['Contractility_ranking'] = usm_adata.obs['Contractility_ranking']

In [ ]:
usm_df

In [ ]:
usm_adata.obs['Contractility_ranking']

In [ ]:
corr_df = usm_df.corrwith(usm_adata.obs['Contractility_ranking'],method='spearman')
corr_df.dropna()

In [ ]:
corr_df = corr_df *-1
corr_df = corr_df.dropna().sort_values()
corr_df.to_csv(f'{output}/usm_contractility_correlation.csv')

In [ ]:
for n in [10,20,50,100]:
    sc.pl.dotplot(usm_adata, corr_df.head(n).index, groupby=['GroupContract'],categories_order=['TL-ND','TNL-GC', 'TNL-BC'],title='Uterine smooth muscle',save=f'positive_w_contractility_{n}_genes.png')
    sc.pl.dotplot(usm_adata, corr_df.sort_values(ascending=False).head(n).index, groupby=['GroupContract'],categories_order=['TL-ND','TNL-GC', 'TNL-BC'],title='Uterine smooth muscle', save=f'negative_w_contractility_{n}_genes.png')

In [ ]:
sc.pl.dotplot(usm_adata, ['DPP6','LINGO2','NOTCH3'], groupby=['GroupContract'],categories_order=['TL-ND','TNL-GC', 'TNL-BC'], title='Uterine smooth muscle',save='sanity_check.png')